In [42]:
import redcap
import pandas as pd
from datetime import timedelta


# REDCap API endpoint and token
data = {
    'token': 'D6807C091B1FC14F57CC9B2972D0B398',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://ruff.mrc.gm:8443/redcap/api/',data['token'])


rand_data=df.export_records(forms=['randomization'])
rand_data=pd.DataFrame(rand_data)

# Specify the fields you want to extract
rand_data=rand_data[['con_participantid_q1', 'ran_date', 'rnd_arm','redcap_event_name']]

rand_data=pd.DataFrame(rand_data)
rand_data = rand_data[
    (rand_data['redcap_event_name'] == 'week_20_arm_1') &
    (rand_data['rnd_arm'] != '')
]

## EXCLUDE ALL WITHDRAWALS FROM THE DATA
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals=df.export_records(forms=['withdrawal'])
df_withdrawals=pd.DataFrame(df_withdrawals)
df_withdrawals=df_withdrawals[['con_participantid_q1','wd_date_q1','wd_reason_q3']]
# Merge the dataframes on 'participant_id'
merged_df = pd.merge(rand_data, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
rand_data= rand_data[~rand_data['con_participantid_q1'].isin(merged_df['con_participantid_q1'])]


# Convert ran_date column to datetime
rand_data['ran_date'] = pd.to_datetime(rand_data['ran_date'])
#Function to calculate due date based on weeks
rand_data = rand_data.drop(columns=['due_date'], errors='ignore')


def calculate_due_date(row):
    # days difference from randomization data to today
    days_elapsed = (datetime.now() - row['ran_date']).days
    # remaining days to complete a week
    rem_days = days_elapsed % 7
    # remaining days to the due date
    to_day = 7 - rem_days if pd.notna(row['ran_date']) else 0
    due_date = row['ran_date'] + timedelta(days=days_elapsed) + timedelta(days=to_day)
    return due_date

# Apply the function to create a new column 'due_date'
rand_data['due_date'] = rand_data.apply(calculate_due_date, axis=1)

df_rand_data = rand_data[['con_participantid_q1', 'ran_date', 'due_date']]


import datetime

# Get the current date
current_date = datetime.date.today()

# Calculate today's date
today = pd.Timestamp.today()


# Find the next Monday
next_monday = today + datetime.timedelta(days=(0 - current_date.weekday() + 7) % 7)

# Convert target_due_date to a datetime object
next_monday = pd.to_datetime(next_monday)

# Calculate the target due date as the next Friday
target_due_date = next_monday + datetime.timedelta(days=4)

# Convert target_due_date to a datetime object
target_due_date = pd.to_datetime(target_due_date)

df_rand_data = df_rand_data[df_rand_data['due_date'] <= target_due_date]



# Get today's date and time
now = datetime.datetime.now()

# Format the current date and time
current_datetime = now.strftime('%d%m%Y_%H%M%S')

# Create the filename with the current date and time
filename = f'Weekly Mobidity CallList_{current_datetime}.csv'

#Save the filtered DataFrame to a CSV file
df_rand_data.to_csv(filename, index=False)

ValueError: Cannot set a DataFrame with multiple columns to the single column due_date